In [1]:
import logging
import pandas as pd
import numpy as np

In [2]:
from data_cleaning import DataCleaning
from data_extraction import DataExtractor
from database_utils import DatabaseConnector

logging.basicConfig(filename='pipeline.log', encoding='utf-8', level=logging.DEBUG,
                    format="%(asctime)s [%(levelname)s] %(name)s - %(funcName).20s - %(message)s",)
logger = logging.getLogger(__name__)


def setup_database(filename):
    db_conn = DatabaseConnector()
    db_credentials = db_conn.read_db_creds(filename)
    engine = db_conn.init_db_engine(db_credentials)
    return db_conn, engine

In [3]:
logger.info('****************************** Starting pipeline ******************************')
db_extractor = DataExtractor()
cleaner = DataCleaning()
source_db, source_engine = setup_database(filename='config/db_creds.yaml')

# with source_engine.execution_options(isolation_level='AUTOCOMMIT').connect() as conn:
#     source_db.list_db_tables(source_engine)  # ['legacy_store_details', 'legacy_users', 'orders_table']
#     extracted = db_extractor.read_rds_table(conn, 'legacy_users')
#     extracted.info()

# df_users = cleaner.clean_user_data(df=extracted)

# Milestone 2 Step 8 - Write users dataframe to target database
target_db, target_engine = setup_database(filename='config/db_creds_target.yaml')
# target_db.upload_to_db(target_engine, df=df_users, table_name='dim_users')

# Task 4 Step 2 - Extract card details to dataframe
pdf_path = 'https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf'
df_card_details = db_extractor.retrieve_pdf_data(pdf_path=pdf_path)
df_card_details = cleaner.clean_card_data(df=df_card_details)

In [6]:
df_card_details = cleaner.clean_card_data(df=df_card_details)

/Users/mashie/src/multinational-retail-data-centralisation/data_cleaning.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month'] = df[column_name].str.slice(5, 7)


In [23]:
df_card_details

,card_number,expiry_date,card_provider,date_payment_confirmed,card_length
0,30060773296197,09/26,Diners Club / Carte Blanche,2015-11-25,14
1,349624180933183,10/23,American Express,2001-06-18,15
2,3529023891650490,06/23,JCB 16 digit,2000-12-26,16
3,213142929492281,09/27,JCB 15 digit,2011-02-12,15
4,502067329974,10/25,Maestro,1997-03-13,12
...,...,...,...,...,...
15304,180036921556789,12/28,JCB 15 digit,1997-06-06,15
15305,180018030448512,11/24,JCB 15 digit,2004-06-16,15
15306,3569953313547220,04/24,JCB 16 digit,2020-02-05,16
15307,4444521712606810,06/27,VISA 16 digit,2008-06-16,16


In [22]:
# df_card_details['card_length'] = df_card_details['card_number'].apply(lambda x: len(x))
df_card_details[(df_card_details['card_provider'] == 'JCB 16 digit')]['card_length'].nunique()
df_card_details[(df_card_details['card_provider'] == 'JCB 16 digit')]['card_length'].nunique()


1